In [1]:
import mediapipe as mp

In [2]:
import numpy as np
import cv2

holistic = mp.solutions.holistic    # utilizes the pose, face and hand landmark models in MediaPipe Pose, MediaPipe Face Mesh and MediaPipe Hands respectively to generate a total of 543 landmarks (33 pose landmarks, 468 face landmarks, and 21 hand landmarks per hand).
hands = mp.solutions.hands   # for showing the visuals 
holis = holistic.Holistic()    #Holistic is the class and holis is its object 
drawing = mp.solutions.drawing_utils   # to show the visuals

#collection of all rows 
X = []
data_size = 0

I0000 00:00:1708754519.777438       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


In [3]:
cap = cv2.VideoCapture(0)   #to get the video feed from our webcam

name = input("enter the name of the data")
while True:
    
    #row
    lst = [] 
    
    
    _, frm = cap.read()   #reading that frame 
    frm = cv2.flip(frm,1)
    
    res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    
    # the res can be null also when it is not able to detect any face 
    
    if res.face_landmarks:   # if someone is there in frame store the landmarks in the list 
        for i in res.face_landmarks.landmark:
            lst.append(i.x - res.face_landmarks.landmark[1].x)
            lst.append(i.y - res.face_landmarks.landmark[1].y)
            
            
    if res.left_hand_landmarks:  # if left hand is visible 
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:   # in left hand there is 21 points so 21*2 for x and y if there is no left hand for those points mark as 0 
        for i in range(42):
            lst.append(0.0)
            
            
    if res.right_hand_landmarks:  #if right hand is visible 
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        for i in range(42):
            lst.append(0.0)
            
    X.append(lst)
    data_size = data_size +1;

    
    # FOR ADDING THE LANDMARKS ON FACE , HANDS 
    drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
    drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
    drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

    cv2.putText(frm, str(data_size), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)    #to display text on the video feed -- here it is data size (50,50) is the location, then it is the type of font, font size, the color, and the thicknesss
    
    cv2.imshow("window", frm)   #showing that it to user
    
    if cv2.waitKey(1) == 27 or data_size > 99:  #we say we have collected 100 samples of data 
        cv2.destroyAllWindows()
        cap.release()
        break

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


enter the name of the datarocknroll


In [4]:
np.save(f"{name}.npy", np.array(X))   #we are storing one particular emoji for 100 samples
print(np.array(X).shape)

(100, 1020)
